# Complete Solution

This notebook that analysis the Crown.

---------------

##### Imports

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import numpy as np
import open3d as o3d

from tree import analyse_tree
import utils.o3d_utils as o3d_utils
import utils.ahn_utils as ahn_utils

##### Load data

In [ ]:
# Load point cloud data. source can be: ahn / cyclo / sonarski
source = 'cyclo'
treecode = '121913_487434'
las_file = '../datasets/'+source+'/'+source+'_'+treecode+'.las'
pcd = o3d_utils.read_las(las_file)

# surface reconstruction
ground_cloud = o3d_utils.read_las('../datasets/ahn_surf/ahn_surf_'+treecode+'.laz')

##### Experimental

In [ ]:
adTree_exe = '../../AdTree-single/build/bin/AdTree.app/Contents/MacOS/AdTree'

# Analyse tree
result = analyse_tree(pcd, ground_cloud, adTree_exe, filter_leaves='curvature', crown_model_method='Convex_Hull')

In [ ]:
o3d.visualization.draw_geometries([result['LoD']])